# Imports and set up

Python 3.10 is required!

In [1]:
OUTPUT_PATH = 'zakl2022.p'

In [2]:
from bs4 import BeautifulSoup

import os
import string
import re
from pprint import pprint
import pickle
from typing import List

from parser_dataclasses import QuestionDraft, CleanedQuestion

In [3]:
def get_digits(input_string: str) -> List[int]:
    cleaned = input_string.translate(str.maketrans('', '', string.punctuation))
    digits = [int(s) for s in cleaned.split() if s.isdigit()]
    return digits

# Get the files

In [4]:
with open(os.path.join('unpacked', 'text.html')) as f:
    soup = BeautifulSoup(f, 'html.parser')

with open('unpacked.html') as f:
    titles_soup = BeautifulSoup(f, 'html.parser')

# Parse titles

In [5]:
questions = {}
part_num = 0
number = 0

contents_table = titles_soup.find('div', {'id': 'outline'}).find('ul')
parts = contents_table.findChildren('li', recursive=False)

for part in parts:
    if 'Часть' in part.findChildren('a', recursive=False)[0].text:
        links_to_questions = part.findChildren('ul', recursive=False)[0].find_all('a', {'class': 'l'})
        for link in links_to_questions:

            new_num = get_digits(link.text)[0]
            if new_num == 1:
                part_num += 1
                number = new_num    
            elif new_num == number + 1:
                number = new_num
            else:
                raise ValueError('Numbers of the questions are not right!')

            questions[(part_num, number)] = QuestionDraft(re.sub('Вопрос\ \d+\.\ ', '', link.text), '', part_num, number)

# Parse questions themselves

In [6]:
questions = {}

for i in range(1, 41):
    questions[(1, i)] = QuestionDraft('', '', 1, i)

for i in range(1, 29):
    questions[(2, i)] = QuestionDraft('', '', 2, i)

for i in range(1, 12):
    questions[(3, i)] = QuestionDraft('', '', 3, i)

for i in range(1, 19):
    questions[(4, i)] = QuestionDraft('', '', 4, i)

for i in range(1, 6):
    questions[(5, i)] = QuestionDraft('', '', 5, i)

In [7]:
pages = []
for page in soup.find_all('div'):
    if page.get('id').startswith('page') and page.get('id').endswith('-div'):
        pages.append(page)

In [8]:
number = 0
part_num = 0

for page in pages:
    pars = page.findChildren('p', recursive=False)
    for par in pars:

        if par.text.startswith('Часть'):
            new_part_num = get_digits(par.text)[0]

            if new_part_num != part_num + 1:
                raise ValueError('Wrong part order!')
            
            if part_num:
                questions[(part_num, number)].text = current_str
            
            part_num = new_part_num
            number = 0
            current_str = ''
            
            continue

        if part_num:

            if get_digits(par.text.strip()):
                probable_new_num = get_digits(par.text.strip())[0]
                beginning = par.text.strip().removeprefix(str(probable_new_num))
                if probable_new_num == number + 1 and beginning.startswith('.'):
                    if not number == 0:
                        try:
                            questions[(part_num, number)].text = current_str
                        except KeyError:
                            print(f'Question number {number}, part {part_num} not found!')
                            print(current_str)
                            print('---')
                
                    current_str = beginning.removeprefix('.')
                    number += 1
                    continue

            current_str += par.text


In [9]:
cleaned_questions = {}

for key, question in questions.items():

    splitted = re.split('(\s[а-яА-Я1-9]\)\s[а-яА-Я1-9a-zA-Z])', question.text)
    cleaned_text = ' '.join(splitted[0].split())

    dividers = splitted[1::2]
    variants = splitted[2::2]
    variants = [' '.join(re.split('\s', (v + d))) for v, d in zip(dividers, variants)]
    
    cleaned_questions[key] = CleanedQuestion(question, cleaned_text, variants)

In [10]:
pickle.dump(cleaned_questions, open(OUTPUT_PATH, 'wb'))
%cd ..
! cp pdfparsing/{OUTPUT_PATH} main/db_pickles
%cd -

/home/dreamtim/Desktop/Coding/Pats/bioquest
/home/dreamtim/Desktop/Coding/Pats/bioquest/pdfparsing


In [11]:
for cleaned_question in cleaned_questions.values():
    if not cleaned_question.answer_variants:
        print(cleaned_question.question.part, cleaned_question.question.number)
        print(cleaned_question.text)
        print('---')

1 24
На схеме показано расположение элементов речевого аппарата человека вмомент произнесения звука: а) [н’]; б) [у]; в) [п]; г) [т].
---
4 3
[3 балла] На рисунках изображены два семязачатка. На каждом из них буквами и цифрами отмечены одни и те же структуры: женский гаметофит, интегумент, микропиле, нуцеллус, ткань материнского спорофита, яйцеклетка. Укажите соответствие между цифрами и буквами. Цифры 1 2 3 4 5 6 Буквы Биология, ЗЭ-2021, 11 класс 40
---
4 4
[6 баллов] Процесс опыления у цветковых растений подразумевает перенос пыльцы от пыльника на рыльце пестика и обычно осуществляется при помощи насекомых. Все, что вы видите на микрофотографиях, имеет непосредственное отношение к реализации насекомоопыления. Установите соответствие между изображениями (1 – 12) и объектами (А – Д), с которыми они связаны в своем развитии. А – Андроцей (Тычинки); Б – Гинецей (пестик); В – Насекомое-опылитель; Г – Околоцветник; Д – Не имеет отношения к процессу опыления. Фотография 1 2 3 4 5 6 7 8 9 10